<h2>IMPORTING THE REQUIRED LIBRARIES</h2>

In [1]:
from termcolor import colored

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from math import sqrt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

<h2>DEFINING THE MAX COLUMNS TO SHOW ATTRIBUTE FROM PANDAS AS NONE</h2>

In [2]:
pd.set_option('display.max_columns', None)

<h2>IMPORTING THE DATA FOR TRAINING AND FOR GENERATE PREDICTIONS INTO DATAFRAMES</h2>

In [3]:
def data(path_training, path_testing):

    dataframe_training = pd.read_csv(path_training).drop(['Unnamed: 0', 'ID'], axis = 1)
    dataframe_testing = pd.read_csv(path_testing).drop(['Unnamed: 0', 'sofifa_id'], axis = 1)

    return dataframe_training, dataframe_testing

fifa_21_base_data, fifa_21_predictions_data = data('data/fifa_data_for_training.csv', 'data/fifa_data_for_predictions.csv')

<h2>TRANSFORMING THE FORMAT OF THE VALUE COLUMN FROM CATEGORICAL TO NUMERICAL USING REGEX<h2>

In [4]:
# https://stackoverflow.com/questions/39684548/convert-the-string-2-90k-to-2900-or-5-2m-to-5200000-in-pandas-dataframe

fifa_21_base_data['Value'] = fifa_21_base_data['Value'].apply(lambda x: x.replace('€', ''))
fifa_21_base_data['Value'] = (fifa_21_base_data['Value'].replace(r'[KM]+$', '', regex = True).astype(float) * fifa_21_base_data['Value'].str.extract(r'[\d\.]+([KM]+)', expand = False).fillna(1).replace(['K', 'M'], [10**3, 10**6]).astype(int))

<h2>MATCHING THE FORMAT OF THE GROWTH AND THE POTENTIAL COLUMNS FROM THE TRAINING AND THE PREDICTIONS DATAFRAMES</h2>

In [5]:
fifa_21_base_data['Growth'] = fifa_21_base_data['Growth'] + fifa_21_base_data['OVA']

<h2>OBTAINING THE INFORMATION ABOUT THE SHAPE OF THE TRAINING DATAFRAME</h2>

In [6]:
def shape(dataframe):

    rows = dataframe.shape[0]
    columns = dataframe.shape[1]

    return rows, columns

fifa_21_base_data_total_rows, fifa_21_base_data_total_columns = shape(fifa_21_base_data)

print(f'\nSHAPE OF THE FIFA 21 BASE DATA')
print(f'------------------------------\n')

print(f'Number of rows: {fifa_21_base_data_total_rows}')
print(f'Number of columns: {fifa_21_base_data_total_columns}')


SHAPE OF THE FIFA 21 BASE DATA
------------------------------

Number of rows: 13700
Number of columns: 100


<h2>OBTAINING THE INFORMATION ABOUT THE SHAPE OF THE PREDICTIONS DATAFRAME</h2>

In [7]:
fifa_21_predictions_data_total_rows, fifa_21_predictions_data_total_columns = shape(fifa_21_predictions_data)

print(f'\nSHAPE OF THE FIFA 21 PREDICTIONS DATA')
print(f'-------------------------------------\n')

print(f'Number of rows: {fifa_21_predictions_data_total_rows}')
print(f'Number of columns: {fifa_21_predictions_data_total_columns}')


SHAPE OF THE FIFA 21 PREDICTIONS DATA
-------------------------------------

Number of rows: 1000
Number of columns: 102


<h2>NORMALIZING THE NAME OF THE COLUMNS OF THE TRAINING DATA</h2>

In [8]:
def normalize_columns(dataframe):

    normalized_columns = list()

    for column in dataframe.columns:

        column = column.lower().replace(' ', '_')
        normalized_columns.append(column)

    dataframe.columns = normalized_columns

    return dataframe

fifa_21_base_data = normalize_columns(fifa_21_base_data)

<h2>ISOLATING THE TARGET FEATURE FROM THE TRAINING DATAFRAME</h2>

In [9]:
y = fifa_21_base_data['ova']

<h2>RENAMING CRUCIAL COLUMNS IN THE PREDICTIONS DATA TO MATCH COLUMNS IN THE TRAINING DATA</h2>

In [10]:
fifa_21_predictions_data = fifa_21_predictions_data.rename(columns = {'mentality_composure': 'composure',
                                                                       'movement_reactions': 'reactions',
                                                                       'pace': 'pac', 'shooting': 'sho',
                                                                       'passing': 'pas', 'dribbling': 'dri',
                                                                       'physic': 'phy', 'value_eur': 'value',
                                                                       'potential': 'growth'})

<h2>OBTAINING ONLY THE COLUMNS OF THE TRAINING DATAFRAME THAT ARE PRESENT IN THE PREDICTION DATAFRAME</h2>

In [11]:
def matching_columns(dataframe_training, dataframe_testing):

    testing_columns = dataframe_testing.columns.to_list()
    matching_columns = list()

    for column in dataframe_training.columns:
        
        if column in testing_columns:
            matching_columns.append(column)

    return dataframe_training[matching_columns], dataframe_testing[matching_columns]

fifa_21_base_data, fifa_21_predictions_data = matching_columns(fifa_21_base_data, fifa_21_predictions_data)

<h2>DROPPING COLUMNS THAT ARE USELESS IN TERMS OF PREDICTING THE OVA FEATURE</h2>

In [12]:
columns_to_drop = ['club', 'joined', 'nationality']

def drop_columns(dataframe, columns_to_drop):
    
    dataframe = dataframe.drop(columns = columns_to_drop)

    return dataframe

fifa_21_base_data = drop_columns(fifa_21_base_data, columns_to_drop)
fifa_21_predictions_data = drop_columns(fifa_21_predictions_data, columns_to_drop)

<h2>CHECKING THE NAN VALUES STATUS FOR THE TRAINING DATA</h2>

In [13]:
def nan_values(dataframe):

    if dataframe.isna().values.any():
        return colored('\nEXISTING NAN VALUES INSIDE THE DATAFRAME', 'red')
    
    else:
        return colored('\nNON EXISTING NAN VALUES INSIDE THE DATAFRAME', 'green')

print(nan_values(fifa_21_base_data))


EXISTING NAN VALUES INSIDE THE DATAFRAME


<h2>OBTAINING THE COLUMNS WITH EXISTING NAN VALUES OF THE TRAINING DATAFRAME</h2>

In [14]:
print(f'\nCOLUMNS WITH NAN VALUES')
print(f'-----------------------\n')

for column in fifa_21_base_data.columns:

    if fifa_21_base_data[column].isna().sum() > 0:

        nan_fifa_column_sum = fifa_21_base_data[column].isna().sum()
        nan_fifa_column_mean = fifa_21_base_data[column].isna().mean()

        print(f'- {column} -> {nan_fifa_column_sum} - {round((nan_fifa_column_mean * 100), 2)} %')


COLUMNS WITH NAN VALUES
-----------------------

- composure -> 329 - 2.4 %


<h2>FILLING THE EXISTING NAN VALUES IN THE TRAINING DATAFRAME</h2> 

In [15]:
def fill_nan_values(dataframe_transformer, dataframe_target):

    for column in dataframe_target.columns:

        if dataframe_target[column].isna().sum() > 0:

            if pd.api.types.is_numeric_dtype(dataframe_target[column]):
                dataframe_target[column] = dataframe_target[column].fillna(dataframe_transformer[column].mean())

            else:
                dataframe_target[column] = dataframe_target[column].fillna(dataframe_transformer[column].mode()[0])

    return dataframe_target

fifa_21_base_data = fill_nan_values(fifa_21_base_data, fifa_21_base_data)

<h2>RE-CHECKING THE NAN VALUES STATUS FOR THE TRAINING DATA</h2>

In [16]:
print(nan_values(fifa_21_base_data))


NON EXISTING NAN VALUES INSIDE THE DATAFRAME


<h2>OBTAINING THE COLUMNS WITH EXISTING NAN VALUES OF THE PREDICTIONS DATAFRAME</h2>

In [17]:
print(f'\nCOLUMNS WITH NAN VALUES')
print(f'-----------------------\n')

for column in fifa_21_predictions_data.columns:

    if fifa_21_predictions_data[column].isna().sum() > 0:

        nan_fifa_column_sum = fifa_21_predictions_data[column].isna().sum()
        nan_fifa_column_mean = fifa_21_predictions_data[column].isna().mean()

        print(f'- {column} -> {nan_fifa_column_sum} - {round((nan_fifa_column_mean * 100), 2)} %')


COLUMNS WITH NAN VALUES
-----------------------

- defending -> 119 - 11.9 %
- gk_diving -> 881 - 88.1 %
- gk_handling -> 881 - 88.1 %
- gk_kicking -> 881 - 88.1 %
- gk_positioning -> 881 - 88.1 %
- gk_reflexes -> 881 - 88.1 %
- pac -> 119 - 11.9 %
- sho -> 119 - 11.9 %
- pas -> 119 - 11.9 %
- dri -> 119 - 11.9 %
- phy -> 119 - 11.9 %
- ls -> 119 - 11.9 %
- st -> 119 - 11.9 %
- rs -> 119 - 11.9 %
- lw -> 119 - 11.9 %
- lf -> 119 - 11.9 %
- cf -> 119 - 11.9 %
- rf -> 119 - 11.9 %
- rw -> 119 - 11.9 %
- lam -> 119 - 11.9 %
- cam -> 119 - 11.9 %
- ram -> 119 - 11.9 %
- lm -> 119 - 11.9 %
- lcm -> 119 - 11.9 %
- cm -> 119 - 11.9 %
- rcm -> 119 - 11.9 %
- rm -> 119 - 11.9 %
- lwb -> 119 - 11.9 %
- ldm -> 119 - 11.9 %
- cdm -> 119 - 11.9 %
- rdm -> 119 - 11.9 %
- rwb -> 119 - 11.9 %
- lb -> 119 - 11.9 %
- lcb -> 119 - 11.9 %
- cb -> 119 - 11.9 %
- rcb -> 119 - 11.9 %
- rb -> 119 - 11.9 %


<h2>DROP COLUMNS WITH TOO MANY NAN VALUES FROM TRAINING AND PREDICTIONS DATAFRAMES</h2>

In [18]:
nan_columns_to_drop = ['gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_positioning']

def drop_columns(dataframe, columns_to_drop):
    
    dataframe = dataframe.drop(columns = columns_to_drop)

    return dataframe

fifa_21_base_data = drop_columns(fifa_21_base_data, nan_columns_to_drop)
fifa_21_predictions_data = drop_columns(fifa_21_predictions_data, nan_columns_to_drop)

<h2>TRANSFORMING THE + AND - COLUMNS IN NUMERICAL FROM THE TRAINING AND THE PREDICTIONS DATAFRAMES</h2>

In [19]:
def categorical_to_numerical(value):

    if pd.isna(value):
        return np.NaN

    if '+' in value:
        values = value.split('+')
        value = int(values[0]) + int(values[1])
    
    else:
        values = value.split('-')
        value = int(values[0]) - int(values[1])

    return value

for column in fifa_21_base_data.iloc[:, 11:].columns:
    
    fifa_21_base_data[column] = fifa_21_base_data[column].apply(lambda value: categorical_to_numerical(value))
    fifa_21_predictions_data[column] = fifa_21_predictions_data[column].apply(lambda value: categorical_to_numerical(value))

<h2>FILLING THE NAN VALUES REMAINING INSIDE THE PREDICTIONS DATAFRAME</h2>

In [20]:
fifa_21_predictions_data = fill_nan_values(fifa_21_base_data, fifa_21_predictions_data)

<h2>RE-CHECKING THE NAN VALUES STATUS FOR THE PREDICTIONS DATA</h2>

In [21]:
print(nan_values(fifa_21_predictions_data))


NON EXISTING NAN VALUES INSIDE THE DATAFRAME


<h2>PERFORMING THE TRAIN TEST SPLIT IN THE TRAINING DATA<h2>

In [22]:
def split_train_test(dataframe_training, target):

    X_train, X_test, y_train, y_test = train_test_split(dataframe_training, target, test_size = 0.2, random_state = 42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_train_test(fifa_21_base_data, y)

<h2>SCALING THE VALUES FOR ALL THE X DATAFRAMES USING THE STANDARD SCALER FUNCTION</h2>

In [23]:
def scale_data(training_x, testing_x, predictions_x):

    scaler = StandardScaler().fit(training_x)

    training_x_array = scaler.transform(training_x)
    training_x_scaled = pd.DataFrame(data = training_x_array, columns = training_x.columns)

    testing_x_array = scaler.transform(testing_x)
    testing_x_scaled = pd.DataFrame(data = testing_x_array, columns = testing_x.columns)

    predictions_x_array = scaler.transform(predictions_x)
    predictions_x_scaled = pd.DataFrame(data = predictions_x_array, columns = predictions_x.columns)

    return training_x_scaled, testing_x_scaled, predictions_x_scaled

X_train_scaled, X_test_scaled, fifa_21_predictions_data_scaled = scale_data(X_train, X_test, fifa_21_predictions_data)

<h2>CREATING AND FITTING A LINEAR REGRESSION MODEL TO PREDICT THE OVA COLUMN</h2>

In [24]:
def create_fit_model(x_training, y_training, selected_model):

    model = selected_model
    model.fit(x_training, y_training)

    return model

linear_regression = create_fit_model(X_train_scaled, y_train, LinearRegression())

<h2>EXTRACTING THE METRICS FROM THE MODEL USING THE TEST DATA FROM THE TRAINING DATAFRAME</h2>

In [25]:
def model_metrics(model, testing_x, testing_y):

    predictions = model.predict(testing_x)

    score = model.score(testing_x, testing_y)
    r2 = r2_score(predictions, testing_y)
    mse = mean_squared_error(predictions, testing_y)
    mae = mean_absolute_error(predictions, testing_y)
    rmse = sqrt(mse)

    return score, r2, mse, mae, rmse

score, r2, mse, mae, rmse = model_metrics(linear_regression, X_test_scaled, y_test)

<h2>RESULTS OF THE MODEL IN THE TEST DATA FROM THE TRAINING DATAFRAME</h2>

In [26]:
print('\nRESULTS AND METRICS OF THE MODEL')
print('--------------------------------\n')

print(f'SCORE: {round(score, 2)}')
print(f'R2 SCORE: {round(r2, 2)}')
print(f'MEAN SQUARED ERROR: {round(mse, 2)}')
print(f'MEAN ABSOLUTE ERROR: {round(mae, 2)}')
print(f'ROOT MEAN SQUARED ERROR: {round(rmse, 2)}')


RESULTS AND METRICS OF THE MODEL
--------------------------------

SCORE: 0.93
R2 SCORE: 0.92
MEAN SQUARED ERROR: 3.61
MEAN ABSOLUTE ERROR: 1.48
ROOT MEAN SQUARED ERROR: 1.9


<h2>EXTRACTING THE METRICS OF THE MODEL USING REAL AND TOTALLY UNSEEN DATA</h2>

In [27]:
fifa_21_predictions_data_real_results = [52, 62, 57, 59, 66, 65, 62, 73, 70, 62, 58, 72, 52, 68, 69, 67, 71, 61, 72, 70, 73, 64, 71, 56, 76, 56, 68, 63, 66, 67, 62, 70, 64, 62, 66, 70, 70, 66, 66, 70, 57, 55, 64, 64, 51, 58, 62, 60, 68, 67, 68, 71, 66, 56, 65, 60, 71, 72, 67, 75, 63, 67, 78, 66, 61, 69, 73, 64, 64, 67, 78, 64, 60, 58, 69, 68, 68, 82, 71, 54, 68, 70, 56, 69, 64, 62, 70, 69, 65, 79, 63, 61, 62, 63, 60, 63, 66, 71, 64, 68, 74, 71, 62, 65, 74, 57, 66, 71, 64, 64, 65, 65, 49, 75, 62, 67, 63, 75, 61, 69, 61, 72, 65, 60, 65, 61, 66, 67, 65, 70, 65, 64, 76, 76, 62, 67, 62, 70, 70, 60, 60, 68, 54, 71, 69, 72, 64, 66, 64, 67, 59, 59, 71, 56, 85, 79, 72, 65, 57, 73, 76, 67, 53, 66, 65, 67, 72, 75, 63, 70, 58, 66, 69, 88, 67, 62, 63, 72, 68, 69, 62, 64, 54, 75, 61, 58, 67, 77, 59, 49, 64, 68, 70, 72, 55, 62, 70, 77, 65, 81, 55, 61, 59, 70, 59, 64, 58, 64, 69, 63, 56, 58, 64, 67, 50, 67, 64, 51, 68, 56, 54, 64, 67, 60, 70, 78, 66, 72, 72, 66, 70, 76, 65, 62, 73, 68, 61, 72, 58, 56, 71, 60, 73, 65, 65, 74, 73, 67, 69, 65, 73, 69, 71, 74, 66, 74, 75, 79, 72, 69, 64, 66, 65, 72, 63, 65, 67, 69, 78, 51, 65, 75, 60, 67, 69, 64, 57, 63, 76, 64, 72, 59, 73, 65, 70, 80, 72, 59, 88, 61, 69, 61, 65, 69, 66, 70, 66, 60, 67, 68, 75, 62, 52, 69, 65, 81, 86, 72, 72, 66, 52, 56, 78, 62, 71, 78, 58, 60, 68, 64, 73, 53, 75, 68, 60, 77, 66, 73, 63, 71, 67, 65, 80, 77, 70, 66, 69, 65, 53, 52, 74, 65, 65, 67, 69, 70, 60, 59, 69, 68, 54, 82, 68, 75, 69, 71, 70, 79, 67, 58, 79, 64, 57, 68, 68, 66, 66, 73, 64, 81, 60, 69, 52, 59, 57, 68, 67, 55, 72, 76, 75, 64, 74, 65, 67, 59, 65, 66, 72, 73, 51, 67, 64, 62, 66, 68, 52, 56, 64, 70, 78, 71, 59, 69, 61, 62, 66, 64, 66, 67, 61, 72, 66, 64, 70, 70, 54, 74, 68, 64, 69, 65, 62, 76, 63, 66, 61, 72, 69, 76, 65, 76, 61, 52, 80, 67, 63, 60, 68, 66, 67, 59, 67, 72, 60, 51, 62, 81, 71, 69, 56, 67, 68, 69, 63, 65, 69, 62, 65, 71, 67, 66, 61, 73, 61, 51, 62, 61, 75, 65, 76, 68, 69, 65, 62, 64, 64, 73, 75, 71, 69, 58, 67, 60, 52, 65, 58, 77, 50, 80, 70, 68, 66, 69, 69, 60, 69, 61, 68, 80, 77, 67, 65, 74, 66, 65, 68, 78, 62, 80, 60, 87, 83, 68, 54, 67, 68, 60, 67, 61, 64, 59, 66, 73, 72, 58, 73, 72, 60, 53, 71, 65, 74, 73, 56, 75, 71, 64, 64, 66, 64, 70, 73, 78, 75, 53, 79, 73, 63, 67, 56, 73, 62, 54, 67, 63, 71, 69, 74, 74, 76, 68, 68, 64, 58, 62, 60, 63, 68, 68, 66, 75, 54, 70, 74, 62, 60, 67, 74, 73, 74, 55, 79, 67, 60, 68, 64, 50, 75, 63, 72, 57, 65, 66, 71, 59, 63, 57, 55, 68, 57, 67, 73, 52, 66, 68, 67, 56, 70, 69, 66, 63, 73, 65, 51, 61, 61, 78, 68, 65, 55, 64, 60, 62, 66, 67, 53, 67, 72, 64, 52, 65, 59, 70, 65, 79, 67, 75, 65, 61, 77, 63, 53, 61, 71, 69, 78, 48, 69, 63, 53, 67, 64, 76, 76, 60, 63, 66, 62, 67, 66, 67, 69, 68, 78, 62, 74, 72, 72, 65, 63, 59, 71, 68, 67, 70, 71, 65, 62, 58, 84, 68, 63, 62, 71, 68, 62, 78, 82, 67, 72, 79, 68, 69, 68, 60, 61, 76, 69, 72, 66, 68, 77, 62, 57, 66, 53, 62, 63, 63, 59, 74, 70, 72, 61, 66, 82, 69, 58, 70, 65, 69, 72, 67, 66, 76, 70, 71, 71, 66, 77, 59, 74, 68, 68, 83, 67, 63, 63, 69, 64, 67, 63, 64, 62, 61, 48, 72, 53, 49, 59, 77, 74, 67, 66, 60, 74, 58, 91, 71, 60, 84, 73, 68, 71, 66, 54, 65, 64, 64, 63, 69, 75, 71, 68, 62, 70, 65, 79, 57, 65, 65, 70, 66, 61, 61, 74, 59, 54, 59, 64, 75, 50, 66, 63, 69, 70, 66, 63, 57, 66, 76, 54, 73, 68, 62, 64, 62, 64, 67, 70, 75, 68, 57, 70, 64, 63, 51, 74, 80, 72, 65, 72, 53, 65, 77, 72, 63, 69, 57, 69, 65, 65, 67, 70, 79, 69, 55, 73, 66, 61, 77, 68, 66, 70, 73, 71, 70, 72, 66, 66, 64, 60, 67, 63, 58, 51, 61, 71, 65, 80, 75, 74, 64, 69, 62, 73, 65, 66, 72, 64, 68, 62, 56, 78, 78, 70, 73, 52, 68, 61, 72, 61, 60, 67, 69, 79, 66, 65, 76, 66, 74, 63, 71, 66, 71, 66, 54, 70, 71, 65, 64, 68, 66, 71, 66, 79, 64, 78, 64, 54, 70, 59, 59, 76, 70, 78, 65, 61, 68, 71, 63, 65, 67, 71, 64, 65, 62, 77, 48, 75, 67, 75, 68, 64, 67, 62, 64, 59, 64, 73, 58, 85, 63, 65, 62, 69, 72, 75, 59, 71, 55, 70, 70, 64, 66, 61, 64, 54, 72, 54, 66, 62, 63, 76, 69, 69, 73, 67, 74, 71, 56, 71, 63, 67, 68, 65, 80, 60, 58, 65, 75, 66, 70, 63, 69, 61, 69, 72, 67, 65, 67, 70, 72, 68, 67, 73, 72, 59, 68]
score_f, r2_f, mse_f, mae_f, rmse_f = model_metrics(linear_regression, fifa_21_predictions_data_scaled, fifa_21_predictions_data_real_results)

<h2>RESULTS OF THE MODEL USING REAL AND UNSEEN DATA</h2>

In [28]:
print('\nRESULTS AND METRICS OF THE MODEL IN REAL DATA')
print('---------------------------------------------\n')

print(f'SCORE: {round(score_f, 2)}')
print(f'R2 SCORE: {round(r2_f, 2)}')
print(f'MEAN SQUARED ERROR: {round(mse_f, 2)}')
print(f'MEAN ABSOLUTE ERROR: {round(mae_f, 2)}')
print(f'ROOT MEAN SQUARED ERROR: {round(rmse_f, 2)}')


RESULTS AND METRICS OF THE MODEL IN REAL DATA
---------------------------------------------

SCORE: 0.91
R2 SCORE: 0.9
MEAN SQUARED ERROR: 4.21
MEAN ABSOLUTE ERROR: 1.6
ROOT MEAN SQUARED ERROR: 2.05
